In [ ]:
from IPython.display import Image
import tensorflow as tf
#import preprocessing as pre
from functools import partial
import matplotlib.pyplot as plt
import numpy as np
import time
import os

In [ ]:
print(tf.__version__)

In [ ]:
# This section need not be edited (It is intended to setup the working environment for tensorflow)

In [ ]:
def config_gpu():
    #Configure Gpus
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
        # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)

In [ ]:
config_gpu()

In [ ]:
#Checkpoint 1: Define Training/Test data size and dimension and explain the rationale for your selection------------------------

Training/Testing size:

Minimum height is 496, minimum length is 384

Image size needs to be smaller than 384

In [ ]:
import cv2

In [ ]:
shapes = []
for folder in os.listdir('../input/mydata/Data'):
    path = '../input/mydata/Data/' + folder
    for file in os.listdir(path):
        l,b,_=cv2.imread(path +'/'+ file).shape
        shapes.append([l,b])


In [ ]:
np.max(shapes,axis = 0)

In [ ]:
batch_size = 32
img_size = 128
val_size = 0.2

In [ ]:
#Preprocessing Images Functions:
def get_label(img_path):
    ls = tf.strings.split(img_path, '/')
    #perform one hot encoding
    label = ls[-2] == ['NORMAL', 'CNV', 'DME', 'DRUNSEN']

    return label

def preprocess_image(img_path, target_size):
    label = get_label(img_path)
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, target_size)

    return img, label

def preprocess_image_2(img_path, target_size, augment = False):
    img, label = preprocess_image(img_path, target_size)
    d3_img = tf.image.grayscale_to_rgb(img)

    if augment == True:
        extra_data = augment_data(d3_img)
        d3_img = extra_data
        label = [label]*len(extra_data)
        
    else:
        pass
    
    return d3_img, label

plt.figure()
#print(preprocess_image_2('../input/mydata/Data/CNV/CNV-103044-1.jpeg', (128,128))[0][:,:,:])
plt.imshow(preprocess_image_2('../input/mydata/Data/CNV/CNV-103044-1.jpeg', (128,128))[0][:,:,:])
plt.show()

In [ ]:
data_path = '../input/mydata/Data'
ls_files = tf.data.Dataset.list_files(data_path + '/*/*')
#augment


val = ls_files.take(int(val_size*len(list(ls_files))))
train = ls_files.skip(int(val_size*len(list(ls_files))))
preprocess_function = partial(preprocess_image, target_size = (img_size, img_size))

val_pipeline = val.map(preprocess_function).shuffle(100).batch(batch_size)
train_pipeline = train.map(preprocess_function).shuffle(100).batch(batch_size)
val_pipeline

In [ ]:
#Checkpoint 2: Print a sample of the data and explain the selected range and the purpose of normalisation----------------------

In [ ]:
image = tf.io.read_file('../input/mydata/Data/DRUNSEN/DRUSEN-2211381-2.jpeg')
image_array = tf.image.decode_jpeg(image).numpy()
#range:
maximum = np.max(image_array)
minimum = np.min(image_array)

print(image_array.shape)
print(image_array.reshape((496,768))[:15,:15])
print('The range of data is {} to {}'.format(minimum, maximum))
print('We can normalize the range to 0 to 1 to reduce the variance')

In [ ]:
#Build Base Model (As defined in the exercise)

In [ ]:
#import keras stuff

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


In [ ]:
model1 = tf.keras.Sequential([
    layers.Conv2D(16, 3, padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation = 'relu'),
    layers.Dense(4, activation = 'sigmoid')
])

model2 = tf.keras.Sequential([
    layers.Conv2D(16, 3, padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation = 'relu'),
    layers.Dense(4, activation = 'sigmoid')
])

In [ ]:
#Define Loss function and Optimiser
loss_func = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

#Checkpoint 4: Explain selection of loss function and optimiser Optimizer

Categorical Crossentropy was chosen because it is a multiclass classification problem

Adam Optimizer was used because it is a state-of-the-art optimizer

In [ ]:
train_loss = tf.keras.metrics.Mean('train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')

test_loss = tf.keras.metrics.Mean('test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [ ]:
#Define Training/Testing Function here
def train_step(model, optimizer, images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_func(labels, predictions)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    
    #backprop
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(tf.argmax(labels, axis = 1), predictions)
    
def test_step(model, images, labels):
    pred = model(images)
    loss = loss_func(labels, pred)
    
    test_loss(pred)
    test_accuracy(tf.argmax(labels,axis = 1), pred)
    

In [ ]:
def plot_graph(ls):
    train_loss, test_loss, train_acc, test_acc = ls
    plt.figure(figsize=(8,8))
    plt.plot(train_loss, label = 'training loss')
    plt.plot(test_loss, label = 'test loss')
    plt.plot(train_acc, label = 'training accuracy')
    plt.plot(test_acc, label = 'test accuracy')
    plt.xlabel('epochs')
    plt.ylabel('metric values')
    plt.ylim((0,1.5))
    plt.legend()
    plt.show()

In [ ]:
def train_model(model, epochs, optimizer, training_pipe, testing_pipe):
    plt.figure()

    train_loss_ls = []
    test_loss_ls = []
    train_acc_ls = []
    test_acc_ls = []
    for e in range(epochs):
        for i, (im, lb) in enumerate(training_pipe):
            train_step(model, optimizer, im, lb)
            print(i/len(train_pipeline))
            
        for j, (im, lb) in enumerate(testing_pipe):
            test_step(model, im, lb)
        train_loss_ls.append(train_loss.result())
        test_loss_ls.append(test_loss.result())
        train_acc_ls.append(train_accuracy.result())
        test_acc_ls.append(test_accuracy.result())
        print('Epoch {} - Training Loss: {}, Training Accuracy: {}, Validation Loss: {}, Validation Accuracy: {}'.format(1+e,train_loss.result(), train_accuracy.result(), test_loss.result(), test_accuracy.result()))
        train_accuracy.reset_states()
        train_loss.reset_states()
        test_accuracy.reset_states()
        test_loss.reset_states()
        
    ls_of_metrics = [train_loss_ls, test_loss_ls, train_acc_ls, test_acc_ls]
    plot_graph(ls_of_metrics)
    
    return ls_of_metrics

In [ ]:
#Checkpoint 5: Display training loss-epoch graph

In [ ]:
#Checkpoint 6: Display Accuracy-epoch graph
values = train_model(model1, 20, optimizer, train_pipeline, val_pipeline)
plot_graph(values)

#Checkpoint 7: Define and explain the choice of transfer base model for transfer learning

I used a ResNet-50 model as the base model, as it was trained on images and I think it should be able to extract the features properly. ResNet-50 has shortcut connections within the model and it is good for extending the model's ability to keep improving.

In [ ]:
base_model = tf.keras.applications.ResNet50(input_shape = (img_size,img_size,3), include_top = False, weights = 'imagenet')
base_model.trainable = False
base_model.summary()

In [ ]:
inputs = keras.Input(shape = (img_size,img_size,3))
x = base_model(inputs)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, activation = 'relu')(x)
outputs = keras.layers.Dense(4, activation = 'softmax')(x)

new_model = keras.Model(inputs, outputs)
print(new_model.summary())

In [ ]:
preprocess_function_2 = partial(preprocess_image_2, target_size = (128, 128))
val_pipeline_2 = val.map(preprocess_function_2).shuffle(100).batch(batch_size)
train_pipeline_2 = train.map(preprocess_function_2).shuffle(100).batch(batch_size)

In [ ]:
trf_val = train_model(new_model, 20, optimizer, train_pipeline_2, val_pipeline_2)

In [ ]:
plot_graph(trf_val)

In [ ]:
#Checkpoint 8: Display graph printout of the base model with the OCT image classication extension


In [ ]:
new_model.summary()

In [ ]:
#Checkpoint 9: The training data currently being used is simple and as such transfer learning isn't as advantageous,
#however some observations can be made when comparing the first model with the second, discuss these observation.

The second model was able to produce a much higher accuracy at the start, for example, within 10 epochs, its validation accuracy has reached about 0.74, whereas for the first model, the validation accuracy has only reached about 0.5. However, the accuracy of the second model 


#Checkpoint 10: Discuss why this may not be favourable and the problems it presents. 

Biased datasets are not favourable as the model may take in the frequency of occurrence of a certain class into account and may skew the model predictions to predict less of that class, causing model performance to drop. 

In [ ]:
#3000 images for all except CNV which has 1000

lst = os.listdir('../input/mydata/Data/CNV') # dir is your directory path
len(lst)

#so we can triple the dataset in CNV

In [ ]:
#Checkpoint 11: Show some methods that can be utilised to negate or minimise these effects. 
#Compare the accuracy and explain the pros and cons of these techniques (If any) 

- Image Augmentation can be used to generate more data in the under-represented class, through transformations such as zoom or reflection. Through this, the model can generalize better and give better performance. The pros of this technique is that it can be done rather easily and one does not need to collect more data. The con is that not all images can be augmented in any way and the training accuaracy may not improve as well.

- Add class weights to make the model focus more on the under-represented class by modifying the loss function used.

Below, a combination of image augmentation on the whole dataset and adding class weights are used for the model.

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split = 0.2,
                                                            width_shift_range=0.2,
                                                          height_shift_range = 0.2,
                                                                shear_range = 0.1, 
                                                                horizontal_flip = True,
                                                         )

train_generator = datagen.flow_from_directory('../input/mydata/Data', target_size = (128,128), batch_size=32, subset = 'training', class_mode = 'categorical')
test_generator = datagen.flow_from_directory('../input/mydata/Data', target_size = (128,128), batch_size=32, subset = 'validation', class_mode = 'categorical')


model2.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
hist = model2.fit(train_generator, epochs = 20, validation_data = test_generator, class_weight = {0: 3.0, 1: 1.0, 2: 1.0, 3: 1.0})


In [ ]:
plt.figure()

for key in hist.history.keys():
    plt.plot(hist.history[key], label = key)
plt.legend()
plt.show()